## Language translation with Torchtext!! ## 

Seq2Seq network with torchtext

https://pytorch.org/tutorials/beginner/torchtext_translation.html

In [5]:
import random
from typing import Tuple

import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
import math
import time

import spacy
from spacy.lang.de.examples import sentences 

In [7]:
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

2023-04-28 08:06:06.616118: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 47.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
2023-04-28 08:06:22.643206: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

# de_tokenizer = get_tokenizer('spacy', language='de')

de_tokenizer = spacy.load("de_core_news_sm")
en_tokenizer = spacy.load("en_core_web_sm")
# en_tokenizer = get_tokenizer('spacy', language='en')

def build_vocab(filepath, tokenizer):
    counter = Counter()
    with io.open(filepath, encoding="utf8") as f:
        for string_ in f:
            counter.update(tokenizer(string_))
    
    return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

def data_process(filepaths):
    raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
    raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
    src_max_length = 0
    tgt_max_length = 0
    data = []
    for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
        de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)], dtype=torch.long)
        en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)], dtype=torch.long)
        data.append((de_tensor_, en_tensor_))
        
        if de_tensor_.size(0) > src_max_length:
            src_max_length = de_tensor_.size(0)
            
        if en_tensor_.size(0) > tgt_max_length:
            tgt_max_length = en_tensor_.size(0)
        
    return data, src_max_length, tgt_max_length

train_data, trsl, trtl = data_process(train_filepaths)
val_data, vsl, vtl = data_process(val_filepaths)
test_data, tesl, tetl = data_process(test_filepaths)

TypeError: ignored

In [ ]:
# We calculate the sentence with max_length
candidates_lengths = [trsl, trtl, vsl, vtl, tesl, tetl]
print(max(candidates_lengths))
MAX_LENGTH = max(candidates_lengths) + 2
print(MAX_LENGTH)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
    de_batch, en_batch, en_batch_out = [], [], []
    a = 0
    for (de_item, en_item) in data_batch:
        de_extra = MAX_LENGTH - (de_item.size(0) + 2)
        en_extra = MAX_LENGTH - (en_item.size(0) + 2)
        en_inp_extra = MAX_LENGTH - (en_item.size(0) + 1)
        
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX]), torch.full((de_extra,), PAD_IDX)], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX]), torch.full((en_extra,), PAD_IDX)], dim=0))
        en_batch_out.append(torch.cat([en_item, torch.tensor([EOS_IDX]), torch.full((en_inp_extra,), PAD_IDX)], dim=0)) # Target input 
    de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
    en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
    en_batch_out = pad_sequence(en_batch_out, padding_value=PAD_IDX)
    return de_batch, en_batch, en_batch_out

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)

In [ ]:
german_id_to_text = lambda x: [de_vocab.itos[t] for t in x]

In [ ]:
english_id_to_text = lambda x: [en_vocab.itos[t] for t in x]

## Now, we create the Seq2Seq model ##

The Seq2Seq model is an encoder-decoder model

In the simplest seq2seq decoder we use only last output of the encoder. This last output is sometimes called the context vector as it encodes context from the entire sequence. This context vector is used as the initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and hidden state. The initial input token is the start-of-string <SOS> token, and the first hidden state is the context vector (the encoder’s last hidden state).

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, gru_layers):
        super(EncoderRNN, self).__init__()
        self.gru_layers = gru_layers
        
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, gru_layers)

    def forward(self, x, hidden):
        batch_size = x.size()[0]
#         print("X :", x.shape)
        embedded = self.embedding(x)
#         print("EMBEDD: ", embedded.shape)
        embedded = embedded.view(1, batch_size, -1)
#         print("EMBEDD VIEW: ", embedded.shape)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self, batch_size):
        return torch.zeros(self.gru_layers, batch_size, self.hidden_size, device=device)

### Attention Decoder ###
If only the context vector is passed betweeen the encoder and decoder, that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to “focus” on a different part of the encoder’s outputs for every step of the decoder’s own outputs. First we calculate a set of attention weights. These will be multiplied by the encoder output vectors to create a weighted combination. The result (called attn_applied in the code) should contain information about that specific part of the input sequence, and thus help the decoder choose the right output words.

Calculating the attention weights is done with another feed-forward layer attn, using the decoder’s input and hidden state as inputs. Because there are sentences of all sizes in the training data, to actually create and train this layer we have to choose a maximum sentence length (input length, for encoder outputs) that it can apply to. Sentences of the maximum length will use all the attention weights, while shorter sentences will only use the first few

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, gru_layers, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.gru_layers = gru_layers

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, gru_layers)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, hidden, encoder_outputs):
        batch_size = x.size()[0]
        # Embedding: a simple lookup table that stores embeddings of a fixed dictionary and size.
        # This module is often used to store word embeddings and retrieve them using indices. 
        # The input to the module is a list of indices, and the output is the corresponding word embeddings
#         print("INPUT SHAPE: ", x.shape)
#         print(batch_size)
        embedded = self.embedding(x)
#         print("EMBEDD: ", embedded.shape)
        embedded = embedded.view(1, batch_size, -1)
#         print("EMBEDD VIEW: ", embedded.shape)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        
        # torch.bmm performs a batch matrix-matrix product of matrices stored in input and mat2.
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self, batch_size):
        return torch.zeros(self.gru_layers, batch_size, self.hidden_size, device=device)

### We  create the classic train/evaluate loop ###

For this task, we have something called **teacher_forcing_ratio**, this helps us vary between giving the network the possibility to try to translate using its own previous prediction (no teacher forcing), or we use the known target for it

In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, target_tensor_out, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):

    encoder_hidden = encoder.initHidden(input_tensor.size(1))
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(MAX_LENGTH, encoder.hidden_size, device=device)

    loss = 0
    
    # We run the input sequence through the encoder
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
#         print(encoder_output.shape)
        encoder_outputs[ei] = encoder_output[0, 0]
    
    decoder_input = torch.full((target_tensor.size(1),), BOS_IDX, device=device)
#     decoder_input = torch.tensor([[BOS_IDX], target_tensor.size(1)], device=device)

#     print(decoder_input.shape)
    decoder_hidden = encoder_hidden
    decoded_words = []
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoded_words.append(english_id_to_text(topi[0]))
            loss += criterion(decoder_output, target_tensor_out[di])
            if di + 1 < target_length:
                decoder_input = target_tensor[di + 1]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoded_words.append(english_id_to_text(topi[0]))
            decoder_input = topi.detach()  # detach from history as input
#             print(target_tensor[di].shape)
            loss += criterion(decoder_output, target_tensor_out[di])

#     print(decoded_words)
    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
def evaluate(encoder, decoder, input_tensor):
    with torch.no_grad():
        input_tensor = input_tensor.to(device)
        encoder_hidden = encoder.initHidden(input_tensor.size(1))
        input_length = input_tensor.size(0)

        encoder_outputs = torch.zeros(MAX_LENGTH, encoder.hidden_size, device=device)
        
        # We run the input sequence through the encoder
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
        
#         print(encoder_outputs[:1])
        decoder_input = torch.full((input_tensor.size(1),), BOS_IDX, device=device)
        decoder_hidden = encoder_hidden
        
        decoded_words = []
        decoder_attentions = torch.zeros(MAX_LENGTH, MAX_LENGTH)

        for di in range(MAX_LENGTH):
#             print(decoder_input)
#             print(decoder_input.shape)
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.topk(1)
            if topi.item() == EOS_IDX:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(english_id_to_text(topi))
#             print(decoded_words)
            decoder_input = topi.detach()
          
        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
# Helper methods

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s' % (asMinutes(s))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def trainIters(encoder, decoder, epochs, print_every=50, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    PAD_IDX = en_vocab.stoi['<pad>']
    criterion = nn.NLLLoss(ignore_index=PAD_IDX)

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    
    for epoch in range(0, epochs +1):
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every
        print("-----------------------")
        print("EPOCH: ", epoch, " out of ", epochs, " epochs")
        print("-----------------------")
        for batch_counter, (src, trg, trg_output) in enumerate(train_iter):
            src, trg, trg_output = src.to(device), trg.to(device), trg_output.to(device)
            src_ = src.permute(1,0)
            trg_ = trg.permute(1,0)
            trg_output_ = trg_output.permute(1, 0)
            
            # Uncomment to go 1 by 1 manual unbatch version (preferably simply set batch = 1)
#             for j in range(len(src_)):
#                 loss = train(src_[j].unsqueeze(dim=1), trg_[j].unsqueeze(dim=1), trg_output_[j].unsqueeze(dim=1), encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
#                 print_loss_total += loss
#                 plot_loss_total += loss
                
            loss = train(src, trg, trg_output, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

    #     training_pairs = [tensorsFromPair(random.choice(pairs))
    #                       for i in range(n_iters)]

            batch_counter += 1

            if batch_counter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, batch_counter / len(train_iter)),
                                             batch_counter, batch_counter/ len(train_iter) * 100, print_loss_avg))

            if batch_counter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
hidden_size = 256

encoder1 = EncoderRNN(INPUT_DIM, hidden_size, 4).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, OUTPUT_DIM, 4, dropout_p=0.1).to(device)

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(encoder1):,} trainable parameters')
print(f'The model has {count_parameters(attn_decoder1):,} trainable parameters')

trainIters(encoder1, attn_decoder1, epochs = 5, print_every=50)

In [ ]:
# Randomly evaluate
def evaluateRandomly(encoder, decoder, n=10):
    for _, (src, trg, tgt_output) in enumerate(test_iter):
        for i in range(n):
#             print(src.shape)
            src_ = src.permute(1,0)
            trg_ = trg.permute(1,0)
#             print(src_[i])
#             print(src_[i].unsqueeze(dim=1).shape)

            src_ = src_[i]
            trg_ = trg_[i]
            print('>', german_id_to_text(src_))
            print('=', english_id_to_text(trg_))
            output_words, attentions = evaluate(encoder, decoder, src_.unsqueeze(dim=1))
#             output_sentence = ' '.join(output_words)
            print('<', output_words)
            print('')
        break

In [ ]:
evaluateRandomly(encoder1, attn_decoder1, 1)

In [ ]:
## Machine Translation is hard task!! ##

The dataset we have + the parameters + training time is not really enough to train a machine translation network! 

Implement and test Pytorch’s own Transformer models for machine translation
Analyze based on the output and measurements.
Compare to provided example seq2seq
